In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt 

from group4_banker import Group4Banker

Let 

$$
\pi_w(a \mid x) = a \left ( \frac{\exp(w^\top x)}{\exp(w^\top x) + 1}\right )
$$

be a policy parametrized by $w \in \mathbb{R}^n$, where $\pi_w$ indicates the probability of whether to grant or not grant a loan given some feature vector $x \in \mathbb{R}^n$. The action $a$ is determined according to the expected utility $\mathbb{E}(U)$ and is obtained by
$$
a = \begin{cases}
1 & \text{ if } \mathbb{E}(U) > 0 \\ 
0 & \text{ if } \mathbb{E}(U) \leq 0
\end{cases}
$$

We remark that $\pi_w$ is differentiable by $\nabla_w \pi_w = x\exp(w^\top x) / (\exp(w^\top x)  + 1)^2$, omitting $a$, which makes gradient based optimization a suitable approach to estimate $w$ under some objective. To maximise revenue $U$ and at the same time account for fairness $F$, we seek the parameters $w$ of $\pi_w$ that maximizes

$$
\int_{\Theta} V_\theta(\pi_w) d\beta(\theta); \quad V_\theta(\pi_w) = (1 - \lambda)\mathbb{E}_{\pi_w}(U) - \lambda \mathbb{E}_{\pi_w}(U)(F).
$$

Here, $\lambda$ is a regularisation parameter balancing fairness and utility, $\Theta$ is the set of possible inputs $\theta \sim \Theta$ to a Random Forest (RF) model $\beta$ that predicts the probability $P_\theta$ of a particular outcome $y$ indicating wether a loan is being repaid or not. Moreover, $\mathbb{E}_{\pi_w}(U) = \mathbb{I}(y = a_{\pi_w})$ indicates model accuracy using $a_{\pi_w} = \arg \max_a \pi_w(a \mid x)$ as model predictions. We define the expected fairness $\mathbb{E}(F)$ as

$$
\mathbb{E}_{\pi_w}(U)(F) = \sum_{y, a, z} \pi_w(a \mid x) \left ( P_\theta (a \mid y, z) - P_\theta (a \mid y) \right )^2
$$

where $z \subset x$ denote some sensitive variables among the features. To optimize our objective, we approximate 

$$
\int_{\Theta} V_\theta(\pi_w) d\beta(\theta) \approx \sum_{i=1}^n V_{\theta^{(i)}}(\pi_w),
$$

using Stochastic Gradient Descent (SGD) with automatic differentiation to obtain $\nabla_{w} V_\theta(\pi_w)$. In each iteration of SGD we collect bootstrap samples  $\theta^{(i)}$ from the validation/test set $\Theta$ and predict $P_{\theta^{(i)}}$ using the RF model $\beta(\theta^{(i)})$. Note that to distinguish betwen $P_\theta (a \mid y, z)$ and $P_\theta (a \mid y, z)$ we manage two RF models: one trained using all variables $x$ for in a boostrap sample $\theta$ and another trained using only $x \setminus z$ variables. Thats is, we exclude the sensitive variables $z$ when training the latter RF model.

In [31]:
from typing import Tuple


def decision_makers(df: pd.DataFrame, df_nonsensitive: pd.DataFrame) -> Tuple:
    """
    Args:
        df: Dataset including all variables.
        df_nonsensitive: Dataet excluding the sensitive variables.
    """
        
    # TODO:
    # * Set interest rate etc.
    # * Add w parameter to DM action method (defaults to zero).
    dm_nonsensitive = None
    dm_sensitive = None
    
    return dm_nonsensitive, dm_sensitive

In [30]:
def action(E_U: float) -> int:
    """
    Args:
        E_U: Expected utility.
        
    Returns:
        Indicator for which action to make. 
    """
    
    return int(E_U > 0)

In [ ]:
def sgd_step(action, x, w, P_sensitives, P_non_sensitives, optimizer, lmbda) -> np.ndarray:
    
    # Optimize objective wrt. parameter w.
    with tf.GradientTape() as tape:

        # Just in case. 
        tape.watch(w)
        
        pi_w = None
        
        U = None
        F = None
        
        # NOTE: Maximize V <=> minimize -1 * V.
        dV_dw = optimizer.minimize((lmbda - 1) * U + lmbda * F, [w])
    
    # Cast to numpy so can be mixed with Python objects.
    return w.numpy()

In [ ]:
def oob_estimate(w_i, X, dm_sensitive, dm_nonsensitive, size_oob=None):
    
    if size_oob is None:
        size_oob = int(X.shape[0] * 1 / 3)
    
    # Bootstrap sample from validation/test data.
    idx = np.random.choice(np.arange(X.shape[0]), replace=True, size=size_oob)

    X_sub = X[test_idx]
    y_sub = y[idx]

    actions, P_sensitives, P_non_sensitives = [], [], []
    for x in X_sub:

        P_sensitives.append(dm_sensitive.get_best_action(w=w_i))
        P_non_sensitives.append(dm_nonsensitive.get_best_action(w=w_i))

    # Think we can do with absolute numbers but in case we need probabilities.
    return np.mean(P_sensitives), np.mean(P_non_sensitives)

In [ ]:
def experiment():
    
    lmbda = 1e-3

    _w = 0
    w = tf.Variable([0], dtype=tf.float32)
    
    # Train RF models on training data. 
    dm_nonsensitive, dm_sensitive = decision_makers()

    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4)
    for _ in range(num_epochs):
        
        P_sensitives, P_non_sensitives = oob_estimate(
            
        )
        
        _w = sgd_step(
            w=w, P_sensitives=P_sensitives, P_non_sensitives=P_non_sensitives
        )
        
    return w.numpy()


experiment()

In [15]:
def prep_data():

    features = ['checking account balance', 'duration', 'credit history',
                'purpose', 'amount', 'savings', 'employment', 'installment',
                'marital status', 'other debtors', 'residence time',
                'property', 'age', 'other installments', 'housing', 'credits',
                'job', 'persons', 'phone', 'foreign']

    target = 'repaid'

    df_train = pd.read_csv("../../data/credit/D_train.csv", sep=' ', names=features+[target])
    df_test = pd.read_csv("../../data/credit/D_test.csv", sep=' ', names=features+[target])

    numerical_features = ['duration', 'age', 'residence time', 'installment', 'amount', 'persons', 'credits']
    quantitative_features = list(filter(lambda x: x not in numerical_features, features))
    D_train = pd.get_dummies(df_train, columns=quantitative_features, drop_first=True)
    D_test = pd.get_dummies(df_test, columns=quantitative_features, drop_first=True)
    encoded_features = list(filter(lambda x: x != target, D_train.columns))

    return D_train, D_test, encoded_features, target

In [17]:
D_train, D_test, encoded_features, target = prep_data()

X_train = D_train.loc[:, encoded_features] 
y_train = D_train.loc[:, target] 

model = Group4Banker(optimize=False, random_state=42)
model.set_interest_rate(0.05)
model.fit(X_train, y_train)

X_test = D_test.loc[:, encoded_features] 
y_test = D_test.loc[:, target] 